## Análise da base de dados `Beta` utilizando algoritmos de ML

Neste notebook será analisado o `Beta dataset` utilizando algoritmos de ML para realizar a (1) extração de características, (2) seleção de características e (3) classificação dos dados

### Pontos importantes do dataset

- Frequências estimuladas (total de 40, com a diferença de 0.2 Hz uma da outra): 8.0, 8.2, ..., 15.6, 15.8;
- Taxa de amostragem: 250 Hz

### Analisar os "momentos" em que ocorrem evocação do sinal SSVEP

1. Criar o objeto `MNE` a partir dos dados dados do participante;
2. Aplicar no objeto `MNE` o filtro passa-faixa nos valores de 6 - 18 Hz;
3. Criar cópias do objeto `MNE` com fatias de tempo menores para analisar momentos que ocorrem estimulos ou não (verificar artigo);
    a) 0.0 - 0.5 segundos e 2.5 - 3.0 segundos ocorre apenas ruído;
    b) 0.5 - 2.5 segundos ocorre sinal SSVEP (com ruídos)
4. Com os sinais separados em objetos `MNE`, aplicar a `FFT`, para que seja possível plotar gráficos que contenham (ou não) as informações.
    - Os dados devem ser plotados no domínio da frequência (após a transformada de Fourier). O FFT pode ser realizado pela biblioteca `scipy.fft`.
    - Deve ser observado que as janelas (a) com ruído não aparecerão de fato o sinal SSVEP.

### Extração de características

Uma característica importante de acordo com o artigo base do dataset `BETA` é o *signal-to-noise ratio* (SNR).
São dois tipos de características SNR que podem ser implementadas: SNR de banda estreita (`narrow-SNR`) e SNR de banda larga (`wide-band SNR`).

Uma boa prática, é considerar o ruído das medidas de `SNR`, uma vez que os dados `SSVEP` não estão estimulados durante os períodos de 0 a 0,5 segundos e de 2,5 a 3 segundos. O ruído pode afetar a precisão das medidas de `SNR` e, portanto, é aconselhável levar isso em consideração.

Vamos realizar todos esses cálculos com dados fictícios:

In [31]:
import numpy as np

ch_ideal = ["Pz", "PO3", "PO5", "PO4", "PO6", "POz", "O1", "Oz", "O2"] #canais ideais
ch_names = ["FP1", "FPZ", "FP2", "AF3", "AF4", "F7", "F5", "F3", "F1", "FZ", "F2", "F4", "F6", "F8", "FT7", "FC5", "FC3", "FC1", "FCZ", "FC2", "FC4", "FC6", "FT8", "T7", "C5", "C3", "C1", "CZ", "C2", "C4", "C6", "T8", "M1", "TP7", "CP5", "CP3", "CP1", "CPZ", "CP2", "CP4", "CP6", "TP8", "M2", "P7", "P5", "P3", "P1", "PZ", "P2", "P4", "P6", "P8", "PO7", "PO5", "PO3", "POZ", "PO4", "PO6", "PO8", "CB1", "O1", "OZ", "O2", "CB2"]


data = np.load("./datasets/beta/data1.npy")


labels = np.load("./datasets/beta/labels1.npy")

data.shape,labels.shape

((160, 64, 750), (160,))

In [32]:
import mne

n_channels = 64
sfreq = 250
info = mne.create_info(ch_names, sfreq=sfreq, ch_types=(['eeg'] * len(ch_names)))

ev = {str(value): index  for index, value in enumerate(sorted(set(labels)))}

labels.max(), labels.min()

(15.8, 8.0)

Agora iremos estimar o ruído de fundo, para calcular posteriormente o `narrow SNR` e o `wide-band SNR`. 

In [33]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
events = np.column_stack((
    np.array(range(len(labels))),
    np.zeros(160, dtype=int),
    le.fit_transform(labels))
)


mne_data = mne.EpochsArray(data, info, events, event_id=ev)
filtered_mne_data = mne.EpochsArray(data, info, events, event_id=ev).filter(7.8, 16.0)
mne_data.get_data().shape

Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 7.8 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.80
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.80 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 4.00 Hz (-6 dB cutoff frequency: 18.00 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 10240 out of 10240 | elapsed:    1.5s finished


(160, 64, 750)

In [34]:
import mne
import numpy as np

# define a baseline
base_start = 0
base_end = 125
rest_start = 625
rest_end = 750

sr = 250
target_frequencies = np.arange(8, 16, 0.2) # define as frequências alvo

target_amplitudes = []

base_powers = []
rest_powers = []
snrs = []
max_spectral_values = []
mean_spectral_values = []

for channel_data in mne_data.get_data():
    target_per_channel = []  # guarda as amplitudes para cada frequência alvo

    base_power_channel = []   # guarda o base_power para cada trial
    rest_power_channel = []  # guarda o rest_power para cada trial
    max_spectral_values_channel = []  # guarda o max_spectral_value para cada trial
    mean_spectral_values_channel = []  # guarda o mean_spectral_value para cada trial

    for sign in channel_data:
        fft_result = np.fft.fft(sign)  # FFT
        psd = np.abs(fft_result) ** 2  # PSD    
        frequencies = np.fft.fftfreq(len(fft_result), 1 / sr)  # frequencias

        target_amplitudes_trial = []

        for target_frequency in target_frequencies:
            index = np.argmin(np.abs(frequencies - target_frequency))  # acha o índice da frequência alvo
            amplitude = np.sqrt(psd[index])  # calcula a amplitude da frequência alvo
            target_amplitudes_trial.append(amplitude)  # guarda a amplitude da frequência alvo

        target_per_channel.append(target_amplitudes_trial)  # guarda as amplitudes para cada frequência alvo

        base_power = np.mean(psd[base_start:base_end]) # calcula o base_power
        rest_power = np.mean(psd[rest_start:rest_end]) # calcula o rest_power

        base_power_channel.append(base_power) # guarda o base_power para cada trial
        rest_power_channel.append(rest_power) 

        max_spectral_values_trial = np.max(psd) # calcula o max_spectral_value
        mean_spectral_values_trial = np.mean(psd)

        max_spectral_values_channel.append(max_spectral_values_trial) # guarda o max_spectral_value para cada trial
        mean_spectral_values_channel.append(mean_spectral_values_trial)

    snr = np.mean(base_power_channel) + np.mean(rest_power_channel) # calcula o snr

    # Store base_power, rest_power, snr, max_spectral_value, and mean_spectral_value for the channel
    base_powers.append(base_power_channel)
    rest_powers.append(rest_power_channel)
    snrs.append(snr)
    max_spectral_values.append(max_spectral_values_channel)
    mean_spectral_values.append(mean_spectral_values_channel)

    target_amplitudes.append(target_per_channel)  # Store the amplitudes for each channel


target_amplitudes = np.array(target_amplitudes)
base_powers = np.array(base_powers)
max_spectral_values = np.array(max_spectral_values)
mean_spectral_values = np.array(mean_spectral_values)
rest_powers = np.array(rest_powers)
snrs = np.array(snrs)

snrs

array([54541.8030593 , 89180.5209516 , 63580.52612903, 58396.86699836,
       37904.7322381 , 63922.60044964, 50671.10162841, 47841.98900474,
       50824.72673771, 43213.69696673, 45810.59980063, 46207.03202505,
       42816.83120961, 49189.6046948 , 32935.88762919, 55458.45753401,
       53533.04108616, 61032.01399977, 57023.33921482, 63295.71471488,
       51526.94841751, 54767.60541026, 54703.96049986, 63817.16312976,
       47609.67906004, 78281.95694162, 47895.21826413, 67436.55545053,
       54237.39636954, 79515.08439016, 61966.79049858, 48163.87890113,
       66008.95489261, 48557.72822784, 54862.1531362 , 48165.67535125,
       55786.2966643 , 73973.70269376, 58952.95739982, 63228.79323659,
       63554.90989799, 79999.03391522, 67870.50395082, 52942.078321  ,
       62025.94832984, 37553.55220677, 43059.93764252, 54359.60620405,
       44318.71702178, 43919.98429919, 55838.32084931, 57940.41519829,
       74269.74373502, 60864.21239428, 48836.4728705 , 38550.75750071,
      

Antes de calcular os SNRs, precisamos obter as amplitudes alvo por meio dos dados EEG:

Vamos calcular o SNR de "banda estreita". Pode ser observado pela seguinte equação:

$SNR_{banda\ estreita} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{média das amplitudes nas frequências vizinhas}}\right)$

Já o SNR de banda larga é definido da seguinte forma:

$SNR_{banda\ larga} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{energia total do espectro de amplitude}}\right)$

In [35]:
snr = np.min(snrs)
target_amplitudes_adjusted = np.abs(target_amplitudes - snr) #ajuste da amplitude do sinal
 
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / snr) # razao sinal ruido de banda estreita
print(narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted) # potencia total
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)
print(wide_band_SNR.shape)



data = np.array([narrow_band_SNR, wide_band_SNR]) # array com as duas razoes sinal ruido
print(data.shape)
data = data.swapaxes(0, 1) # troca de eixos
print(data.shape)
data = data.swapaxes(1, 2) # troca de eixos
print(data.shape)
data = data.reshape(data.shape[0], data.shape[1], data.shape[2] * data.shape[3]) # reshape

print(data.shape,max_spectral_values.shape,mean_spectral_values.shape,rest_powers.shape,base_powers.shape)
#salva os dados


np.save("./datasets/beta/data1_filtered.npy", data)

(160, 64, 40)
(160, 64, 40)
(2, 160, 64, 40)
(160, 2, 64, 40)
(160, 64, 2, 40)
(160, 64, 80) (160, 64) (160, 64) (160, 64) (160, 64)


Ao final desta etapa, será obtido um vetor de características. Estas podem ser:
- `narrow SNR` (brigatória);
- `wide-band SNR` (brigatória);
- Maior valor espectral (FFT);
- Média dos valores espectrais (FFT).

Dimensionalidade dos dados será explicada da seguinte forma:

`40, 4, 64, 750` -> 40 targets, 4 trials, 64 canais e 750 valores
`160, 64 (SNR) + 64 (média) + 64 (maior) ...`
Resultando em `160, 192`.

### Seleção de características e classificação

Como existem diversos eletrodos (canais) que não obtém sinal SSVEP, podemos extrair as caracteríscas que não contribuem para a classificação dos dados.

Podemos utilizar o método `RFE` (*Recursive Feature Elimination*) aplicado por meio de `sklearn.feature_selection.RFE`, aprimorando o parâmetro `n_features_to_select` até obter o melhor resultado de classificação.

Para a classificação propriamente dita, é considerado o uso do método `SVM`.